In [17]:
import pandas as pd
from sklearn.naive_bayes import GaussianNB
import csv

airlines=pd.read_csv('airlines.csv')
airports = pd.read_csv('airports.csv')
flights = pd.read_csv('flights.csv')
print(airlines.head())
print(airports.head())
print(flights.head())

  IATA_CODE                 AIRLINE
0        UA   United Air Lines Inc.
1        AA  American Airlines Inc.
2        US         US Airways Inc.
3        F9  Frontier Airlines Inc.
4        B6         JetBlue Airways
  IATA_CODE                              AIRPORT         CITY STATE COUNTRY  \
0       ABE  Lehigh Valley International Airport    Allentown    PA     USA   
1       ABI             Abilene Regional Airport      Abilene    TX     USA   
2       ABQ    Albuquerque International Sunport  Albuquerque    NM     USA   
3       ABR            Aberdeen Regional Airport     Aberdeen    SD     USA   
4       ABY   Southwest Georgia Regional Airport       Albany    GA     USA   

   LATITUDE  LONGITUDE  
0  40.65236  -75.44040  
1  32.41132  -99.68190  
2  35.04022 -106.60919  
3  45.44906  -98.42183  
4  31.53552  -84.19447  
   YEAR  MONTH  DAY  DAY_OF_WEEK AIRLINE  FLIGHT_NUMBER TAIL_NUMBER  \
0  2015      1    1            4      AS             98      N407AS   
1  2015      1   

/Users/apoorvamusunuri/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2871: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [18]:
flights = pd.merge(flights, airlines, how = 'left', left_on = 'AIRLINE', right_on = 'IATA_CODE', sort = False)
flights = pd.merge(flights, airports, how = 'left', )
combined.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE_x,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,IATA_CODE,AIRLINE_y
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,AS,Alaska Airlines Inc.
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,AA,American Airlines Inc.
2,2015,1,1,4,US,840,N171US,SFO,CLT,20,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,US,US Airways Inc.
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,20,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,AA,American Airlines Inc.
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,25,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,AS,Alaska Airlines Inc.


In [21]:
flights.info(memory_usage = 'deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5819079 entries, 0 to 5819078
Data columns (total 31 columns):
YEAR                   int64
MONTH                  int64
DAY                    int64
DAY_OF_WEEK            int64
AIRLINE                object
FLIGHT_NUMBER          int64
TAIL_NUMBER            object
ORIGIN_AIRPORT         object
DESTINATION_AIRPORT    object
SCHEDULED_DEPARTURE    int64
DEPARTURE_TIME         float64
DEPARTURE_DELAY        float64
TAXI_OUT               float64
WHEELS_OFF             float64
SCHEDULED_TIME         float64
ELAPSED_TIME           float64
AIR_TIME               float64
DISTANCE               int64
WHEELS_ON              float64
TAXI_IN                float64
SCHEDULED_ARRIVAL      int64
ARRIVAL_TIME           float64
ARRIVAL_DELAY          float64
DIVERTED               int64
CANCELLED              int64
CANCELLATION_REASON    object
AIR_SYSTEM_DELAY       float64
SECURITY_DELAY         float64
AIRLINE_DELAY          float64
LATE_AIRCRAFT

In [22]:
for dtype in ['float','int','object']:
    selected_dtype = flights.select_dtypes(include=[dtype])
    mean_usage_b = selected_dtype.memory_usage(deep=True).mean()
    mean_usage_mb = mean_usage_b / 1024 ** 2
    print("Average memory usage for {} columns: {:03.2f} MB".format(dtype,mean_usage_mb))

Average memory usage for float columns: 41.78 MB
Average memory usage for int columns: 40.36 MB
Average memory usage for object columns: 206.87 MB


In [ ]:
# We're going to be calculating memory usage a lot,
# so we'll create a function to save us some time!

def mem_usage(pandas_obj):
    if isinstance(pandas_obj,pd.DataFrame):
        usage_b = pandas_obj.memory_usage(deep=True).sum()
    else: # we assume if not a df it's a series
        usage_b = pandas_obj.memory_usage(deep=True)
    usage_mb = usage_b / 1024 ** 2 # convert bytes to megabytes
    return "{:03.2f} MB".format(usage_mb)

flights_int = flights.select_dtypes(include=['int'])
converted_int = flights_int.apply(pd.to_numeric,downcast='unsigned')

print(mem_usage(flights_int))
print(mem_usage(converted_int))

compare_ints = pd.concat([flights_int.dtypes,converted_int.dtypes],axis=1)
compare_ints.columns = ['before','after']
compare_ints.apply(pd.Series.value_counts)

In [25]:
flights_float = flights.select_dtypes(include=['float'])
converted_float = flights_float.apply(pd.to_numeric,downcast='float')

print(mem_usage(flights_float))
print(mem_usage(converted_float))

compare_floats = pd.concat([flights_int.dtypes,converted_float.dtypes],axis=1)
compare_floats.columns = ['before','after']
compare_floats.apply(pd.Series.value_counts)

710.00 MB
355.00 MB


,before,after
float32,NaN,16.0
float64,16.0,NaN


In [27]:
optimized_flights = flights.copy()

optimized_flights[converted_int.columns] = converted_int
optimized_flights[converted_float.columns] = converted_float

print(mem_usage(flights))
print(mem_usage(optimized_flights))

2395.00 MB
1679.00 MB


In [29]:
flights_obj = flights.select_dtypes(include=['object']).copy()
flights_obj.describe()

,AIRLINE,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,CANCELLATION_REASON
count,5819079,5804358,5819079,5819079,89884
unique,14,4897,930,930,4
top,WN,N480HA,ATL,ATL,B
freq,1261855,3768,346836,346904,48851


In [33]:
airline = flights_obj.AIRLINE
print(airline.head())

airline_cat = airline.astype('category')
print(airline_cat.head())

print(airline_cat.head().cat.codes)
print(mem_usage(airline))
print(mem_usage(airline_cat))

0    AS
1    AA
2    US
3    AA
4    AS
Name: AIRLINE, dtype: object
0    AS
1    AA
2    US
3    AA
4    AS
Name: AIRLINE, dtype: category
Categories (14, object): [AA, AS, B6, DL, ..., UA, US, VX, WN]
0     1
1     0
2    11
3     0
4     1
dtype: int8
260.00 MB
5.00 MB


In [34]:
converted_obj = pd.DataFrame()

for col in flights_obj.columns:
    num_unique_values = len(flights_obj[col].unique())
    num_total_values = len(flights_obj[col])
    if num_unique_values / num_total_values < 0.5:
        converted_obj.loc[:,col] = flights_obj[col].astype('category')
    else:
        converted_obj.loc[:,col] = flights_obj[col]

In [36]:
print(mem_usage(flights_obj))
print(mem_usage(converted_obj))

compare_obj = pd.concat([flights_obj.dtypes,converted_obj.dtypes],axis=1)
compare_obj.columns = ['before','after']
compare_obj.apply(pd.Series.value_counts)

1241.00 MB
44.00 MB


,before,after
object,5.0,NaN
category,NaN,1.0
category,NaN,1.0
category,NaN,1.0
category,NaN,1.0
category,NaN,1.0


In [37]:
optimized_flights[converted_obj.columns] = converted_obj

mem_usage(optimized_flights)

'483.00 MB'

In [9]:
gnb = GaussianNB()
test = gnb.fit()

_csv.reader